In [1]:
import psycopg2 #import the PostgreSQL adapter
import pandas as pd #Used for data analysis and manipilation
from sqlalchemy import create_engine #function to vreat a database engine
import numpy as np #For array manipulation
from psycopg2 import OperationalError
import re #python string library

In [2]:
def create_connection(db_name, db_user, db_password, db_host, db_port): #Python function to create a connection to PostgreSQL server. (Reused from Data Warehousing Lab3)
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection


In [3]:
db_name = "Olympic"  #database parameters
db_user = "postgres"
db_password = "postgres"  
db_host = "localhost"  
db_port = "5432"
db_server = "dw_2024"

In [4]:
connection = create_connection(db_name, db_user, db_password, db_host, db_port)  #Create connection to the PostgreSQL server function

Connection to PostgreSQL DB successful


In [5]:
cursor = connection.cursor() #Cursor allow python code to execute PostgreSQL command in a database session

In [6]:
connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}" #connection string to the database
engine = create_engine(connection_string) #Create engine for the database

In [7]:
countries_df = pd.read_csv("./list-of-countries_areas-by-continent-2024.csv", header=None)

In [8]:
countries_df.columns = ["country", "continent"]

In [9]:
hosts_df = pd.read_csv("./olympic_hosts.csv", header=0)

In [10]:
hosts_df.columns = ["hostid", "enddate", "startdate","location","name","season","year"]

In [11]:
lifeexp_df = pd.read_csv("./life-expectancy.csv", header=0)

In [12]:
lifeexp_df.columns= ["country","code","year","lifeexpectancy"]

In [13]:
medals_df = pd.read_csv("./olympic_medals.csv", header=0)

In [14]:
medals_df.columns = ["discipline","year","event","gender","medal","participanttype","participanttitle","url","name","country","code","code3"]

In [15]:
mental_df = pd.read_csv("./mental-illness.csv",header=0)

In [16]:
mental_df.columns = ["country","code","year","depression","schizophrenia","bipolar","eatingdisorder","anxiety"]

In [17]:
economic_df = pd.read_csv("./Economic data.csv",header=0)

In [18]:
economic_df.columns = ["year", "yearcode", "country", "code", "poverty", "gdpcap", "gdpgrowth", "intsrv","mort", "hlthexp", "govhlthcap", "prvhlthcap", "exthlthcap"]

In [19]:
population_df = pd.read_csv("./Global Population.csv",header=0,encoding='ISO-8859-1')

In [20]:
#Data Cleaning

In [21]:
hosts_df['enddate'] = pd.to_datetime(hosts_df['enddate'], format='%Y-%m-%dT%H:%M:%SZ') #Convert the start times to datetime format
hosts_df['startdate'] = pd.to_datetime(hosts_df['startdate'], format='%Y-%m-%dT%H:%M:%SZ') #Convert the end times to datetime format

In [22]:
lifeexp_df = lifeexp_df.dropna() #lifeexpectancy has continents with na country codes which will not be required so they are dropped from the dataframe

In [23]:
medals_df=medals_df.dropna() #mental illness dataframe has rows with no counntry codes that are regions and continents which will be dropped from the dataframe

In [24]:
economic_df['poverty'] = economic_df['poverty'].replace('..', np.nan) #In each column in the economy data frame there are strings ".." for values with no data which will be replace with NaN 
economic_df['gdpcap'] = economic_df['gdpcap'].replace('..', np.nan)
economic_df['gdpgrowth'] = economic_df['gdpgrowth'].replace('..', np.nan)
economic_df['intsrv'] = economic_df['intsrv'].replace('..', np.nan)
economic_df['mort'] = economic_df['mort'].replace('..', np.nan)
economic_df['hlthexp'] = economic_df['hlthexp'].replace('..', np.nan)
economic_df['govhlthcap'] = economic_df['govhlthcap'].replace('..', np.nan)
economic_df['prvhlthcap'] = economic_df['prvhlthcap'].replace('..', np.nan)
economic_df['exthlthcap'] = economic_df['exthlthcap'].replace('..', np.nan)

In [25]:
economic_df=economic_df[0:-5] # Last five rows are invalid rows containing info about the data and spaces in the csv file

In [26]:
population_df = pd.melt(population_df, id_vars=["Population (Millions of people)"], var_name="Year", value_name="Population")

In [27]:
population_df['Population'] = population_df['Population'].replace('no data', np.nan)

In [28]:
population_df.columns = ["country","year","population"]

In [29]:
def extract_year(value):
    year_pattern = r'\b(18|19|20)\d{2}\b'
    match = re.search(year_pattern, value)
    if match:
        return int(match.group(0))  # Convert the matched year to an integer
    else:
        return None 
        
medals_df['year'] = medals_df['year'].apply(extract_year)  #To obtain the year from the title event.

In [30]:
medals_df = medals_df.drop(columns=["code"]) #To drop the coutnry code with only two characters as standard and other tables follow the three letter structure

In [31]:
medals_df = medals_df.rename(columns={"code3":"code"}) #Rename column to code

In [32]:
#Create SQL tables in the PostgreSQL database and upload the data from the dataframes to the tables

In [33]:
cursor.execute("CREATE TABLE countries (country VARCHAR(255) NOT NULL PRIMARY KEY, continent VARCHAR(255) NOT NULL);")

DuplicateTable: relation "countries" already exists


In [34]:
connection.commit()

In [35]:
countries_df.to_sql("countries", con=engine, if_exists="append", index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "countries_pkey"
DETAIL:  Key (country)=(country) already exists.

[SQL: INSERT INTO countries (country, continent) VALUES (%(country__0)s, %(continent__0)s), (%(country__1)s, %(continent__1)s), (%(country__2)s, %(continent__2)s), (%(country__3)s, %(continent__3)s), (%(country__4)s, %(continent__4)s), (%(country__5)s, %(c ... 8878 characters truncated ...  %(continent__232)s), (%(country__233)s, %(continent__233)s), (%(country__234)s, %(continent__234)s)]
[parameters: {'country__0': 'country', 'continent__0': 'region', 'country__1': 'India', 'continent__1': 'Asia', 'country__2': 'China', 'continent__2': 'Asia', 'country__3': 'United States', 'continent__3': 'North America', 'country__4': 'Indonesia', 'continent__4': 'Asia', 'country__5': 'Pakistan', 'continent__5': 'Asia', 'country__6': 'Nigeria', 'continent__6': 'Africa', 'country__7': 'Brazil', 'continent__7': 'South America', 'country__8

In [42]:
cursor.execute("CREATE TABLE economic (year INT NOT NULL, yearcode VARCHAR(255) NOT NULL, country VARCHAR(255) NOT NULL, code VARCHAR(3) NOT NULL,poverty FLOAT,gdpcap FLOAT,gdpgrowth FLOAT,intsrv FLOAT, mort FLOAT, hlthexp FLOAT, govhlthcap FLOAT, prvhlthcap FLOAT, exthlthcap FLOAT);")

In [43]:
connection.commit()

In [44]:
economic_df.to_sql("economic", con=engine, if_exists="append", index=False)

200

In [54]:
cursor.execute("CREATE TABLE population (country VARCHAR(255), year INT NOT NULL, population FLOAT);")

In [55]:
connection.commit()

In [56]:
population_df.to_sql("population", con=engine, if_exists="append", index=False)

318

In [89]:
cursor.execute("ROLLBACK")

In [66]:
cursor.execute("CREATE TABLE lifeexp  (country VARCHAR(255) NOT NULL, code VARCHAR(255) NOT NULL, year INT NOT NULL, lifeexpectancy FLOAT);")

In [67]:
connection.commit()

In [68]:
lifeexp_df.to_sql("lifeexp", con=engine, if_exists="append", index=False)

61

In [76]:
cursor.execute("CREATE TABLE millness  (country VARCHAR(255) NOT NULL, code VARCHAR(255), year INT NOT NULL, depression FLOAT, schizophrenia FLOAT, bipolar FLOAT, eatingdisorder FLOAT, anxiety FLOAT);")

In [77]:
connection.commit()

In [78]:
mental_df.to_sql("millness", con=engine, if_exists="append", index=False)

840

In [81]:
cursor.execute("CREATE TABLE hosts  (hostid VARCHAR(255) NOT NULL PRIMARY KEY, enddate TIMESTAMP WITH TIME ZONE, startdate TIMESTAMP WITH TIME ZONE, location VARCHAR(255), name	 VARCHAR(255), season VARCHAR(255), year INT);")

In [82]:
connection.commit()

In [83]:
hosts_df.to_sql("hosts", con=engine, if_exists="append", index=False)

53

In [94]:
cursor.execute("CREATE TABLE medals (discipline VARCHAR(255), year INT NOT NULL, event VARCHAR(255), gender VARCHAR(255), medal VARCHAR(255), participanttype VARCHAR(255),participanttitle VARCHAR(255), url VARCHAR(255),name VARCHAR(255), country VARCHAR(255), code VARCHAR(255));")

In [96]:
connection.commit()

In [97]:
medals_df.to_sql("medals",con=engine,if_exists="append", index=False)

667

In [36]:
#Creating the facttable and dimension tables

In [37]:
#For the facttable my columns are factid,country,year,sum of bronze for that year, sum of silver for that year, sum of gold for that year, mental health statistics

In [38]:
medal_sums = medals_df.groupby(['year', 'country', 'medal']).size().unstack(fill_value=0)

In [39]:
medal_sums = medal_sums.reset_index()

In [40]:
years = hosts_df.drop_duplicates()

In [41]:
years = hosts_df["year"]

In [42]:
year = pd.DataFrame(data = hosts_df["year"],columns=["year"])

In [43]:
year = year.drop_duplicates()

In [44]:
year

,year
0,2022
1,2020
2,2018
3,2016
4,2014
5,2012
6,2010
7,2008
8,2006
9,2004


In [45]:
medal_sums.columns = ["year","country","bronze","gold","silver"]

In [46]:
all_years_countries = pd.MultiIndex.from_product([year["year"], countries_df['country']], names=['year', 'country']).to_frame(index=False)

In [47]:
all_years_countries

,year,country
0,2022,country
1,2022,India
2,2022,China
3,2022,United States
4,2022,Indonesia
...,...,...
8690,1896,Montserrat
8691,1896,Falkland Islands
8692,1896,Niue
8693,1896,Tokelau


In [48]:
medal_sums

,year,country,bronze,gold,silver
0,1896,Greece,1,0,1
1,1900,France,2,1,1
2,1900,Great Britain,2,4,0
3,1904,United States of America,6,3,4
4,1908,France,0,2,0
...,...,...,...,...,...
554,2022,Norway,0,2,2
555,2022,People's Republic of China,0,2,0
556,2022,ROC,6,2,4
557,2022,Sweden,2,0,2


In [49]:
medal_sums = pd.merge(all_years_countries, medal_sums, on=['year', 'country'], how='left')

In [50]:
medal_sums = medal_sums.fillna(0)

In [52]:
medal_sums["bronze"] = medal_sums["bronze"].astype(int)

In [54]:
medal_sums["gold"] = medal_sums["gold"].astype(int)

In [55]:
medal_sums["silver"] = medal_sums["silver"].astype(int)

In [58]:
medal_sums = medal_sums[1:]